In [7]:
from data_distillation.models.transformer.feature_extractors.triplet_cross_attention_vit import TripletCrossAttentionViT as TCAiT

import torch.nn as nn
import torch

## Test and debug T-CAiT model

In [8]:
EMBED_DIM = 512
NUM_CLASSES = 21841
NUM_EXTRACTOR_HEADS = 8
NUM_CLASSIFIER_HEADS = 8
BATCH_SIZE = 16
IMG_CHANNELS = 3
IMG_DIM = 224
USE_MINIPATCH = False


In [9]:
# test initialization

model = TCAiT(embed_dim=EMBED_DIM, num_classes=NUM_CLASSES, num_extractor_heads=NUM_EXTRACTOR_HEADS, num_classifier_heads=NUM_CLASSIFIER_HEADS, in_channels=IMG_CHANNELS, in_dim=IMG_DIM, extractor_use_minipatch=USE_MINIPATCH)

In [10]:
# test forward function

anchor = torch.rand(BATCH_SIZE, IMG_CHANNELS, IMG_DIM, IMG_DIM)
positive = torch.rand(BATCH_SIZE, IMG_CHANNELS, IMG_DIM, IMG_DIM)
negative = torch.rand(BATCH_SIZE, IMG_CHANNELS, IMG_DIM, IMG_DIM)

z_anchor, z_positive, z_negative, Y = model(anchor, positive, negative)

In [11]:
print(model)

Extractor
Name                                               | Params       | Size                
------------------------------------------------------------------------------------------
patcher.patch_conv.weight                          |       393216 | (512, 3, 16, 16)    
patcher.patch_conv.bias                            |          512 | (512,)              
anchor_cls_tokenizer.cls_tokens                    |          512 | (1, 1, 512)         
positive_cls_tokenizer.cls_tokens                  |          512 | (1, 1, 512)         
negative_cls_tokenizer.cls_tokens                  |          512 | (1, 1, 512)         
anchor_pos_encoder.pos_embedding                   |       100864 | (1, 197, 512)       
positive_pos_encoder.pos_embedding                 |       100864 | (1, 197, 512)       
negative_pos_encoder.pos_embedding                 |       100864 | (1, 197, 512)       
transformer_blocks.0.norm1.weight                  |          512 | (512,)              
transform